## Introduction

Now you have learnt how to make $N$-body simulations, we can start to find DM halos from the particles. And the most commonly used method for this is called the friends-of-friends (fof) algorithm. The idea is quite simple: if a DM particle $A$ is close enough to any other particles belong to a DM halo $B$, then particle $A$ belongs to halo $B$. You can read more about the friends of friends algorithm at e.g. here: https://swift.dur.ac.uk/docs/FriendsOfFriends/algorithm_description.html

### Goals:
* Run a friends of friends algorithm on a simulation
* Plot particles that are in haloes
* Plot a halo mass function
* Plot displacement fields in Lagrangian space
* Plot halo particles in Lagrangian space

In [ ]:
### Run the following lines in your terminal to install the pyfof package
# git clone https://github.com/simongibbons/pyfof
# cd pyfof/
# pip install --use-pep517 .

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("../../../")

import sheet_unfolding.sim as sim
from matplotlib.colors import LogNorm

import pyfof 

## Run a simulation

First, we need to get the particle distribution by running a simulation. The code you have worked on in the last notebook is implemented in `sim.CosmologicalSimulation2d` so we don't need to write it here again.

In [ ]:
myic = sim.ic.IC2DCosmo(512, L=100., rs=1.5, sigma8=2.)

mysim = sim.CosmologicalSimulation2d(myic, aic=0.02, dafac_max=0.05, da_max=0.02, ngrid_pm=128)

mysim.integrate_till(1.)

In [ ]:
rho = sim.sim.deposit2d(mysim.pos, 128, myic.L, mode="cic")
plt.imshow(np.clip(rho / np.mean(rho), 1e-2, None).T, origin="lower", norm = LogNorm(vmin=1e-2, vmax=1e2), extent=[0,100.,0.,100.])
plt.title("a = %g" % mysim.a)
plt.xlabel(r"$x$ [Mpc/$h$]", fontsize=14)
plt.ylabel(r"$y$ [Mpc/$h$]", fontsize=14)

## Find halos with fof

Now we need to find friends of friends groups with a linking length of 0.2 mean particle separations. Discard all groups which have less than 20 members. Then **make a density plot of particles that are in a group with N>=20 members, and compare it with the density plot of all particles.**

Note: What we are doing here is not exactly correct, because we are neglecting the periodic boundary conditions. However, let's keep it simple for now ;)

In [ ]:
linking_length = 0.2*np.sqrt(np.mean(100.**2/512.**2))
print(linking_length)

In [ ]:
res = pyfof.friends_of_friends(mysim.pos.reshape(-1,2), linking_length)

### Target Output
![title](solutions/img/haloes_versus_particles.png)

# Plot the halo mass function

Plot the fof halo mass function. I.e. the number of objects with a given mass (through a histogram). The mass of a fof-group equals the sum of the masses of the particles in the group. Since all particles have equal mass, you can just multiply the length of the group by the particle mass $m_0$.

In [ ]:
m0 = mysim.mass[0,0]
print(m0) # in the unit of M_sun

### Target Output
![title](solutions/img/haloes_massfunction.png)

## Lagrangian space

Now that you know how to run a simulation, let's explore a bit about seeing the simulation results from a new perspective -- in the Lagrangian space! 

Lagrangian space is basically the space of the initial locations of particles. We started with all particles on a grid (myic.qi[...,0:2]). Since we kept all particles in the same order (i.e. if the initial location of particle $A$ is stored at `myic.qi[i,j]`, then its final location is stored in `mysim.pos[i,j]`), making a Lagrangian plot can be as simple as an plt.imshow(myparticleproperty, ...)

Let's plot the displacement field in Lagrangian space. The displacement field  $\vec{s}$ describes how much a particle moved from its initial location and it is defined so that
\begin{align}
\vec{x}(\vec{q}) = \vec{q} + \vec{s}(\vec{q})
\end{align}

where $\vec{x}$ are the particle positions and $\vec{q}$ their Lagrangian (initial coordinates). Therefore in principle $\vec{s} = \vec{x} - \vec{q}$. However, since we always wrap particle positions periodically between $[0,L]$, you have to undo the periodic wrapping. You can do this by subtracting/adding $L$ so that $\vec{s}$ lies in $[-L/2, L/2]$.

Make a plot of each, the x and y components, of the $\vec{s}$. Further, put a quiver plot on top to indicate the displacement by arrows (To avoid overcrowding the image, in the quiver plot only show the displacement for every 16th particle).

In [ ]:
L = 100.
disp = (((mysim.pos - myic.qi) + L/2.) % L) - L/2.

### Target Output
![title](solutions/img/haloes_lagrangian_displacement.png)

## Lagrangian space haloes

You have seen the DM haloes in Euclidean space, now let's have a look at the Lagrangian space haloes, which basically gives you the info about which group of particles at the initial condition ends up being in the same haloes. 

### Task
Plot the halo index in Lagrangian space, so that different haloes will have different colors. Also plot the displacement field on top as a quiver plot. To get the halo index in Lagrangian space, you will have to mark each particle by the halo index it is part of. For this you will have to rearange a bit the information that you got from the output of pyfof.

### Target Output
![title](solutions/img/haloes_lagrangian_protohaloes.png)